In [1]:
import pandas as pd
import matplotlib as plt
import datetime
import time
import numpy as np
import gc
import sqlite3
import re
import pickle

In [2]:
#pd.set_option('display.height', 1000)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
#pd.set_option('display.width', 1000)

In [3]:
def make_prediction(assetName, date):
    news_con = sqlite3.connect("NewsData.db")
    table_name = re.sub(r'\W+', '', assetName)
    table_name = "News_" + table_name
    query = """SELECT * FROM """ + table_name  + """ WHERE "time" = ?""" 
    news_df = pd.read_sql(sql=query, con=news_con, params=[date])
    news_con.close()
    
    market_con = sqlite3.connect("MarketData.db")
    table_name = re.sub(r'\W+', '', assetName)
    table_name = "Market_" + table_name
    query = """SELECT * FROM """ + table_name  + """ WHERE "time" = ?""" 
    date_time = date + " 22:00:00"
    market_df = pd.read_sql(sql=query, con=market_con, params=[date_time])
    market_con.close()

    range_sent = [ 1, 0, -1]
    range_urg = [1, 3]
    cols_filtered = ['rel_firstMention', 'rel_SentCount', 'relevance', 'firstMentionSentence', 
               'sentenceCount', 'sentimentWordCount', 'wordCount']
    for i in range_sent:
        for j in range_urg:
            for col in cols_filtered:
                new_col = col + "_" + str(j) + '_' + str(i)
                news_df[new_col] = 0.0
                news_df.loc[((news_df['sentimentClass'] == i)  & (news_df['urgency'] == j)),new_col] = \
                    news_df.loc[((news_df['sentimentClass'] == i)  & (news_df['urgency'] == j)),col]
    news_df.drop(labels=cols_filtered + ['urgency','sentimentClass'], axis=1, inplace=True)
    gc.collect()

    news_df['returnsOpenPrevMktres1']  = float(market_df['returnsOpenPrevMktres1'])
    news_df['returnsOpenPrevMktres10'] = float(market_df['returnsOpenPrevMktres10'])
    news_df['returnsOpenPrevMktres1_dir'] = news_df['returnsOpenPrevMktres1'].apply(lambda x: 0 if x<0 else 1)
    news_df['returnsOpenPrevMktres10_dir'] = news_df['returnsOpenPrevMktres10'].apply(lambda x: 0 if x<0 else 1)

    req_feature_columns = ['returnsOpenPrevMktres1_dir', 'returnsOpenPrevMktres10_dir', 'relevance_1_1', 
                           'firstMentionSentence_1_1', 'sentimentWordCount_1_1', 'relevance_3_1',
                           'firstMentionSentence_3_1', 'sentimentWordCount_3_1', 'relevance_1_0',
                           'firstMentionSentence_1_0', 'sentimentWordCount_1_0', 'relevance_3_0',
                           'firstMentionSentence_3_0', 'sentimentWordCount_3_0', 'relevance_1_-1', 
                           'firstMentionSentence_1_-1','sentimentWordCount_1_-1','relevance_3_-1', 
                           'firstMentionSentence_3_-1', 'sentimentWordCount_3_-1', 'rel_SentCount_1_1', 
                           'rel_SentCount_3_1', 'rel_firstMention_1_1', 'rel_firstMention_3_1', 
                           'rel_firstMention_1_0', 'rel_SentCount_1_0', 'rel_firstMention_3_0', 'rel_firstMention_1_-1', 
                           'rel_SentCount_3_0', 'rel_SentCount_1_-1', 'rel_firstMention_3_-1', 'rel_SentCount_3_-1']
    X_data = news_df[req_feature_columns].values
    loaded_model = pickle.load(open('finalized_model.sav', 'rb'))
    Y_predict = loaded_model.predict(X_data)
    X_mean = news_df.groupby(['time','assetName'], as_index=False).mean()[req_feature_columns].values
    Y_predict_mean = loaded_model.predict(X_mean)
    del news_df, market_df
    return Y_predict, Y_predict_mean

In [ ]:
make_prediction("Morgan Stanley", "2009-01-05")

In [ ]:
news_con = sqlite3.connect("NewsData.db")

# Market database

In [4]:
assetName = "Morgan Stanley"
start_date = "2009-01-05"
end_date = "2014-01-05"

In [66]:
def predict_stock(assetName, start_date, end_date):

    market_con = sqlite3.connect("MarketData.db")
    table_name = re.sub(r'\W+', '', assetName)
    table_name = "Market_" + table_name
    query = """SELECT * FROM """ + table_name  + """ WHERE "time" >= ? AND "time" <= ? """ 
    start_date_time = start_date + " 22:00:00"
    end_date_time = end_date + " 22:00:00"
    market_df = pd.read_sql(sql=query, con=market_con, params=[start_date_time, end_date_time])
    market_con.close()

    news_con = sqlite3.connect("NewsData.db")
    table_name = re.sub(r'\W+', '', assetName)
    table_name = "News_" + table_name
    query = """SELECT * FROM """ + table_name  + """ WHERE "time" >= ? AND "time" <= ? """ 
    news_df = pd.read_sql(sql=query, con=news_con, params=[start_date, end_date])
    news_con.close()

    range_sent = [ 1, 0, -1]
    range_urg = [1, 3]
    cols_filtered = ['rel_firstMention', 'rel_SentCount', 'relevance', 'firstMentionSentence', 
               'sentenceCount', 'sentimentWordCount', 'wordCount']
    for i in range_sent:
        for j in range_urg:
            for col in cols_filtered:
                new_col = col + "_" + str(j) + '_' + str(i)
                news_df[new_col] = 0.0
                news_df.loc[((news_df['sentimentClass'] == i)  & (news_df['urgency'] == j)),new_col] = \
                    news_df.loc[((news_df['sentimentClass'] == i)  & (news_df['urgency'] == j)),col]
    news_df.drop(labels=cols_filtered + ['urgency','sentimentClass'], axis=1, inplace=True)
    gc.collect()

    market_df['time'] = pd.to_datetime(market_df['time']).dt.date
    news_df['time'] = pd.to_datetime(news_df['time']).dt.date

    def data_prep(market_df,news_df):
        kcol = ['time']
        news_df = news_df.groupby(kcol, as_index=False).mean()
        market_df = pd.merge(market_df, news_df, how='left', left_on=['time'], right_on=['time'])   
        null_df = market_df[market_df.isna().any(axis=1)]
        market_df = market_df.dropna(axis=0)
        return null_df, market_df

    null_df, market_news = data_prep(market_df, news_df)

    del news_df, market_df

    market_news['returnsOpenPrevMktres1_dir'] = market_news['returnsOpenPrevMktres1'].apply(lambda x: 0 if x<0 else 1)
    market_news['returnsOpenPrevMktres10_dir'] = market_news['returnsOpenPrevMktres10'].apply(lambda x: 0 if x<0 else 1)

    req_feature_columns = ['returnsOpenPrevMktres1_dir', 'returnsOpenPrevMktres10_dir', 'relevance_1_1', 
                               'firstMentionSentence_1_1', 'sentimentWordCount_1_1', 'relevance_3_1',
                               'firstMentionSentence_3_1', 'sentimentWordCount_3_1', 'relevance_1_0',
                               'firstMentionSentence_1_0', 'sentimentWordCount_1_0', 'relevance_3_0',
                               'firstMentionSentence_3_0', 'sentimentWordCount_3_0', 'relevance_1_-1', 
                               'firstMentionSentence_1_-1','sentimentWordCount_1_-1','relevance_3_-1', 
                               'firstMentionSentence_3_-1', 'sentimentWordCount_3_-1', 'rel_SentCount_1_1', 
                               'rel_SentCount_3_1', 'rel_firstMention_1_1', 'rel_firstMention_3_1', 
                               'rel_firstMention_1_0', 'rel_SentCount_1_0', 'rel_firstMention_3_0', 'rel_firstMention_1_-1', 
                               'rel_SentCount_3_0', 'rel_SentCount_1_-1', 'rel_firstMention_3_-1', 'rel_SentCount_3_-1']

    X_data = market_news[req_feature_columns].values
    Y_predict = loaded_model.predict(X_data)
    market_news['predicted'] = Y_predict
    market_news['expected'] = market_news['returnsOpenNextMktres10'].map(lambda x: 0 if x<0 else 1)
    return_col = ['time', 'volume', 'close', 'open','predicted','expected']
    market_news = market_news[return_col]

    null_df['expected'] = null_df['returnsOpenNextMktres10'].map(lambda x: 0 if x<0 else 1)
    null_df['predicted'] = null_df['expected']
    null_df = null_df[return_col]

    final_df = pd.concat([market_news, null_df]).sort_values(['time'])
    final_df['average'] = (final_df['open'] + final_df['close'])/2.0
    del null_df, market_news
    gc.collect()
    return final_df


In [67]:
predict_stock(assetName, start_date, end_date)

,time,volume,close,open,predicted,expected,average
0,2009-01-05,25452042.0,17.62,16.8000,0,1,17.21000
1,2009-01-06,38951702.0,19.58,18.0700,0,1,18.82500
2,2009-01-07,30916751.0,18.10,19.0700,1,1,18.58500
3,2009-01-08,21116269.0,18.82,17.9000,1,1,18.36000
4,2009-01-09,35861704.0,19.06,18.9700,1,1,19.01500
5,2009-01-12,49198602.0,18.79,20.0800,1,1,19.43500
6,2009-01-13,31391567.0,18.86,18.2700,1,1,18.56500
7,2009-01-14,39581552.0,17.19,18.2500,1,1,17.72000
8,2009-01-15,41843063.0,16.26,17.1000,1,1,16.68000
9,2009-01-16,42346767.0,15.59,16.9800,1,1,16.28500


In [5]:
market_con = sqlite3.connect("MarketData.db")
table_name = re.sub(r'\W+', '', assetName)
table_name = "Market_" + table_name
query = """SELECT * FROM """ + table_name  + """ WHERE "time" >= ? AND "time" <= ? """ 
start_date_time = start_date + " 22:00:00"
end_date_time = end_date + " 22:00:00"
market_df = pd.read_sql(sql=query, con=market_con, params=[start_date_time, end_date_time])
market_con.close()

In [19]:
market_df.shape

(1259, 9)

In [6]:
news_con = sqlite3.connect("NewsData.db")
table_name = re.sub(r'\W+', '', assetName)
table_name = "News_" + table_name
query = """SELECT * FROM """ + table_name  + """ WHERE "time" >= ? AND "time" <= ? """ 
news_df = pd.read_sql(sql=query, con=news_con, params=[start_date, end_date])
news_con.close()

1490

In [11]:
range_sent = [ 1, 0, -1]
range_urg = [1, 3]
cols_filtered = ['rel_firstMention', 'rel_SentCount', 'relevance', 'firstMentionSentence', 
           'sentenceCount', 'sentimentWordCount', 'wordCount']
for i in range_sent:
    for j in range_urg:
        for col in cols_filtered:
            new_col = col + "_" + str(j) + '_' + str(i)
            news_df[new_col] = 0.0
            news_df.loc[((news_df['sentimentClass'] == i)  & (news_df['urgency'] == j)),new_col] = \
                news_df.loc[((news_df['sentimentClass'] == i)  & (news_df['urgency'] == j)),col]
news_df.drop(labels=cols_filtered + ['urgency','sentimentClass'], axis=1, inplace=True)
gc.collect()

168

In [12]:
#market_col = [ 'time','returnsOpenNextMktres10', 'returnsOpenPrevMktres1','returnsOpenPrevMktres10']
#market_df = market_df[market_col]
market_df['time'] = pd.to_datetime(market_df['time']).dt.date
news_df['time'] = pd.to_datetime(news_df['time']).dt.date


In [23]:
news_df.groupby('time', as_index=False).mean().shape

(1490, 44)

In [39]:
def data_prep(market_df,news_df):
    kcol = ['time']
    news_df = news_df.groupby(kcol, as_index=False).mean()
    market_df = pd.merge(market_df, news_df, how='left', left_on=['time'], right_on=['time'])   
    null_df = market_df[market_df.isna().any(axis=1)]
    market_df = market_df.dropna(axis=0)
    return null_df, market_df

In [40]:
null_df, market_news = data_prep(market_df, news_df)

In [41]:
null_df

,index_x,time,assetName,volume,close,open,returnsOpenNextMktres10,returnsOpenPrevMktres1,returnsOpenPrevMktres10,index_y,rel_firstMention_1_1,rel_SentCount_1_1,relevance_1_1,firstMentionSentence_1_1,sentenceCount_1_1,sentimentWordCount_1_1,wordCount_1_1,rel_firstMention_3_1,rel_SentCount_3_1,relevance_3_1,firstMentionSentence_3_1,sentenceCount_3_1,sentimentWordCount_3_1,wordCount_3_1,rel_firstMention_1_0,rel_SentCount_1_0,relevance_1_0,firstMentionSentence_1_0,sentenceCount_1_0,sentimentWordCount_1_0,wordCount_1_0,rel_firstMention_3_0,rel_SentCount_3_0,relevance_3_0,firstMentionSentence_3_0,sentenceCount_3_0,sentimentWordCount_3_0,wordCount_3_0,rel_firstMention_1_-1,rel_SentCount_1_-1,relevance_1_-1,firstMentionSentence_1_-1,sentenceCount_1_-1,sentimentWordCount_1_-1,wordCount_1_-1,rel_firstMention_3_-1,rel_SentCount_3_-1,relevance_3_-1,firstMentionSentence_3_-1,sentenceCount_3_-1,sentimentWordCount_3_-1,wordCount_3_-1
498,510732,2010-12-27,Morgan Stanley,5368261.0,27.51,27.25,0.009896,-0.001557,-0.016590,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1001,348645,2012-12-26,Morgan Stanley,9056353.0,18.88,19.02,0.039817,0.010044,0.105104,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1058,447040,2013-03-20,Morgan Stanley,17819327.0,22.74,22.59,-0.075112,-0.014857,-0.024126,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1160,619773,2013-08-14,Morgan Stanley,7220991.0,26.79,26.91,-0.007392,0.005270,-0.018328,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1253,773412,2013-12-26,Morgan Stanley,5408746.0,31.10,31.03,0.006335,-0.005825,-0.027749,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1256,778288,2013-12-31,Morgan Stanley,8458176.0,31.36,30.89,0.022141,-0.006446,-0.037397,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1258,781532,2014-01-03,Morgan Stanley,7811091.0,31.51,31.17,0.046663,0.001894,-0.003649,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [24]:
market_news['returnsOpenPrevMktres1_dir'] = market_news['returnsOpenPrevMktres1'].apply(lambda x: 0 if x<0 else 1)
market_news['returnsOpenPrevMktres10_dir'] = market_news['returnsOpenPrevMktres10'].apply(lambda x: 0 if x<0 else 1)

In [25]:
 req_feature_columns = ['returnsOpenPrevMktres1_dir', 'returnsOpenPrevMktres10_dir', 'relevance_1_1', 
                           'firstMentionSentence_1_1', 'sentimentWordCount_1_1', 'relevance_3_1',
                           'firstMentionSentence_3_1', 'sentimentWordCount_3_1', 'relevance_1_0',
                           'firstMentionSentence_1_0', 'sentimentWordCount_1_0', 'relevance_3_0',
                           'firstMentionSentence_3_0', 'sentimentWordCount_3_0', 'relevance_1_-1', 
                           'firstMentionSentence_1_-1','sentimentWordCount_1_-1','relevance_3_-1', 
                           'firstMentionSentence_3_-1', 'sentimentWordCount_3_-1', 'rel_SentCount_1_1', 
                           'rel_SentCount_3_1', 'rel_firstMention_1_1', 'rel_firstMention_3_1', 
                           'rel_firstMention_1_0', 'rel_SentCount_1_0', 'rel_firstMention_3_0', 'rel_firstMention_1_-1', 
                           'rel_SentCount_3_0', 'rel_SentCount_1_-1', 'rel_firstMention_3_-1', 'rel_SentCount_3_-1']

In [26]:
X_data = market_news[req_feature_columns].values
loaded_model = pickle.load(open('finalized_model.sav', 'rb'))

D:\Program Files\Miniconda3\envs\cse6242\lib\site-packages\sklearn\base.py:251: UserWarning: Trying to unpickle estimator LabelEncoder from version 0.19.1 when using version 0.20.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [27]:
    Y_predict = loaded_model.predict(X_data)

In [28]:
Y_predict.shape

(1252,)

In [50]:
market_news['predicted'] = Y_predict

In [51]:
market_news['expected'] = market_news['returnsOpenNextMktres10'].map(lambda x: 0 if x<0 else 1)

In [52]:
return_col = ['time', 'assetName', 'volume', 'close', 'open','predicted','expected']

In [53]:
market_news = market_news[return_col]

In [54]:
null_df['expected'] = null_df['returnsOpenNextMktres10'].map(lambda x: 0 if x<0 else 1)
null_df['predicted'] = null_df['expected']
null_df = null_df[return_col]

In [59]:
final_df = pd.concat([market_news, null_df]).sort_values(['time'])

In [60]:
final_df.tail()

,time,assetName,volume,close,open,predicted,expected
1254,2013-12-27,Morgan Stanley,4116365.0,31.06,31.12,0,0
1255,2013-12-30,Morgan Stanley,6201517.0,30.85,31.07,0,1
1256,2013-12-31,Morgan Stanley,8458176.0,31.36,30.89,1,1
1257,2014-01-02,Morgan Stanley,9843424.0,31.03,31.33,0,1
1258,2014-01-03,Morgan Stanley,7811091.0,31.51,31.17,1,1


In [ ]:
table_name = re.sub(r'\W+', '', assetName)
table_name = "News_" + table_name

In [ ]:
query = """SELECT * FROM """ + table_name  + """ WHERE "time" = ?""" 
news_df = pd.read_sql(sql=query, con=news_con, params=["2009-01-05"])

In [ ]:
news_con.close()

In [ ]:
market_con = sqlite3.connect("MarketData.db")

In [ ]:
table_name = re.sub(r'\W+', '', assetName)
table_name = "Market_" + table_name

In [ ]:
query = """SELECT * FROM """ + table_name  + """ WHERE "time" = ?""" 
market_df = pd.read_sql(sql=query, con=market_con, params=["2009-01-05 22:00:00"])

In [ ]:
market_con.close()

In [ ]:
market_df

In [ ]:
news_df

In [ ]:
range_sent = [ 1, 0, -1]
range_urg = [1, 3]
# print("Unique Sentiment Values : {}".format(range_sent))
# print("Unique Urgency Values:  {}".format(range_urg))

cols_filtered = ['rel_firstMention', 'rel_SentCount', 'relevance', 'firstMentionSentence', 
           'sentenceCount', 'sentimentWordCount', 'wordCount']
for i in range_sent:
    for j in range_urg:
        for col in cols_filtered:
            new_col = col + "_" + str(j) + '_' + str(i)
            news_df[new_col] = 0.0
            news_df.loc[((news_df['sentimentClass'] == i)  & (news_df['urgency'] == j)),new_col] = \
                news_df.loc[((news_df['sentimentClass'] == i)  & (news_df['urgency'] == j)),col]
news_df.drop(labels=cols_filtered + ['urgency','sentimentClass'], axis=1, inplace=True)
gc.collect()
print("Given data Shape : {}".format(news_df.shape))

In [ ]:
news_df

In [ ]:
news_df['returnsOpenPrevMktres1']  = float(market_df['returnsOpenPrevMktres1'])
news_df['returnsOpenPrevMktres10'] = float(market_df['returnsOpenPrevMktres10'])

In [ ]:
news_df['returnsOpenPrevMktres1_dir'] = news_df['returnsOpenPrevMktres1'].apply(lambda x: 0 if x<0 else 1)
news_df['returnsOpenPrevMktres10_dir'] = news_df['returnsOpenPrevMktres10'].apply(lambda x: 0 if x<0 else 1)

In [ ]:
req_feature_columns = [ 'returnsOpenPrevMktres1_dir', 'returnsOpenPrevMktres10_dir', 
                   'relevance_1_1', 'firstMentionSentence_1_1', 
        'sentimentWordCount_1_1', 
       'relevance_3_1', 'firstMentionSentence_3_1', 'sentimentWordCount_3_1',
       'relevance_1_0', 'firstMentionSentence_1_0', 
       'sentimentWordCount_1_0', 'relevance_3_0',
       'firstMentionSentence_3_0', 'sentimentWordCount_3_0', 
       'relevance_1_-1', 'firstMentionSentence_1_-1',
       'sentimentWordCount_1_-1',
       'relevance_3_-1', 
       'firstMentionSentence_3_-1', 
       'sentimentWordCount_3_-1', 
        'rel_SentCount_1_1', 'rel_SentCount_3_1', 'rel_firstMention_1_1', 'rel_firstMention_3_1', 
        'rel_firstMention_1_0', 'rel_SentCount_1_0', 'rel_firstMention_3_0', 'rel_firstMention_1_-1', 
        'rel_SentCount_3_0', 'rel_SentCount_1_-1', 'rel_firstMention_3_-1', 'rel_SentCount_3_-1']

In [ ]:
X_data = news_df[req_feature_columns].values

In [ ]:
loaded_model = pickle.load(open('finalized_model.sav', 'rb'))

In [ ]:
Y_predict = loaded_model.predict(X_data)

In [ ]:
X_mean = news_df.groupby(['time','assetName'], as_index=False).mean()[req_feature_columns].values

In [ ]:
Y_predict_mean = loaded_model.predict(X_mean)

In [ ]:
Y_predict_mean

# Rough Work

In [ ]:
table_name

In [ ]:
X_mean

In [34]:
 df = pd.DataFrame({'age': [5, 6, np.NaN],
...                    'born': [pd.NaT, pd.Timestamp('1939-05-27'),
...                             pd.Timestamp('1940-04-25')],
...                    'name': ['Alfred', 'Batman', ''],
...                    'toy': [None, 'Batmobile', 'Joker']})
>>> df

,age,born,name,toy
0,5.0,NaT,Alfred,None
1,6.0,1939-05-27,Batman,Batmobile
2,NaN,1940-04-25,,Joker


In [37]:
df.isna().any(axis=1)

0     True
1    False
2     True
dtype: bool

In [38]:
df[df.isna()]

,age,born,name,toy
0,NaN,NaT,NaN,None
1,NaN,NaT,NaN,NaN
2,NaN,NaT,NaN,NaN
